# Generate Profiles
This notebook generates load profiles, grid import profiles, and generator profiles for the PyPSA hybrid renewable project.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

## Configuration

In [ ]:
# Configuration parameters
START_DATE = "2024-01-01 00:00:00"
HOURS = 24
DAYS = 7
TIMES = pd.date_range(START_DATE, periods=HOURS * DAYS, freq="h", tz="UTC")
TIME_ZONE = "UTC+00"
POWER_FACTOR = 0.9  # Applies to all load types
BASE_VOLTAGE = 0.415  # kV (415V system)

## Load Profiles

In [ ]:
# Create directory structure
output_dir = Path("../hybrid_renewable_pypsa/data/profiles/load_profiles")
results_dir = Path("../hybrid_renewable_pypsa/results")
output_dir.mkdir(parents=True, exist_ok=True)
results_dir.mkdir(parents=True, exist_ok=True)

def add_daily_pattern(base_load, daily_curve, noise_level=0.1):
    """Add daily pattern with noise"""
    daily = base_load * daily_curve
    noise = np.random.normal(0, noise_level * base_load, len(daily))
    return np.clip(daily + noise, 0, None)

def generate_load_profile(base, daily_curve, noise_level):
    """Generate generic load profile"""
    p_set = add_daily_pattern(base, daily_curve, noise_level)
    q_set = p_set * np.tan(np.arccos(POWER_FACTOR))
    return np.tile(p_set, DAYS), np.tile(q_set, DAYS)

# Load profile definitions
PROFILE_DEFINITIONS = {
    "residential": (0.2, np.array([0.6, 0.5, 0.4, 0.4, 0.5, 0.6, 1.0, 1.2,
                                    0.8, 0.7, 0.6, 0.6, 0.7, 0.8, 0.9, 1.0,
                                    1.4, 1.6, 1.8, 2.0, 1.6, 1.4, 1.2, 1.0]), 0.15),
    "commercial": (0.3, np.array([0.3, 0.3, 0.3, 0.3, 0.4, 0.5, 0.8, 1.0,
                                   1.2, 1.4, 1.5, 1.6, 1.6, 1.7, 1.7, 1.6,
                                   1.4, 1.2, 0.8, 0.6, 0.4, 0.3, 0.3, 0.3]), 0.1),
    "industrial": (0.4, np.array([0.8, 0.8, 0.8, 0.8, 1.0, 1.0, 1.0, 1.0,
                                   1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2,
                                   1.0, 1.0, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8]), 0.05)
}

PROFILE_METADATA = {
    "residential_1": "Residential area 1",
    "residential_2": "Residential area 2",
    "residential_3": "Suburban housing",
    "residential_4": "Urban apartments",
    "commercial_1": "Commercial complex A",
    "commercial_2": "Shopping district",
    "commercial_3": "Office park",
    "industrial_1": "Factory zone 1",
    "industrial_2": "Factory zone 2",
    "industrial_3": "Processing plant"
}

# Generate profiles and metadata
metadata_records = []
profiles = {
    **{f"residential_{i+1}": generate_load_profile(*PROFILE_DEFINITIONS["residential"]) for i in range(4)},
    **{f"commercial_{i+1}": generate_load_profile(*PROFILE_DEFINITIONS["commercial"]) for i in range(3)},
    **{f"industrial_{i+1}": generate_load_profile(*PROFILE_DEFINITIONS["industrial"]) for i in range(3)}
}

# Save profiles and collect metadata
for profile_id, (p_set, q_set) in profiles.items():
    df = pd.DataFrame({
        "time": TIMES.strftime("%Y-%m-%d %H:%M:%S"),
        "p_set": np.round(p_set, 4),
        "q_set": np.round(q_set, 4)
    })

    csv_header = (f"# time [{TIME_ZONE}], p_set [MW], q_set [MVAR] "
                  f"(power_factor={POWER_FACTOR})\n")
    file_path = output_dir / f"{profile_id}.csv"
    file_path.write_text(csv_header + df.to_csv(index=False))

    metadata_records.append({
        "profile_id": profile_id,
        "load_type": profile_id.split("_")[0],
        "voltage_kV": BASE_VOLTAGE,
        "time_zone": TIME_ZONE,
        "power_factor": POWER_FACTOR,
        "data_source": "synthetic",
        "description": PROFILE_METADATA[profile_id]
    })

# Save metadata file
metadata_file = output_dir / "load_profiles_metadata.csv"
pd.DataFrame(metadata_records).to_csv(metadata_file, index=False)

print("Successfully generated:")
print(f"- {len(profiles)} load profiles in {output_dir}")
print(f"- Metadata file: {metadata_file}")


# Plot load profiles
plt.figure(figsize=(12, 6))
for profile_id, (p_set, q_set) in profiles.items():
    plt.plot(TIMES, p_set, label=f"{profile_id} (P)")
    plt.plot(TIMES, q_set, label=f"{profile_id} (Q)", linestyle='--')
plt.title("Load Profiles")
plt.xlabel("Time")
plt.ylabel("Power (MW/MVAR)")
plt.legend()
plt.grid(True)

# Save plot
plot_path = results_dir / "load_profiles.png"
plt.savefig(plot_path)

plt.show()

print(f"Plot saved to {plot_path}")

## Grid Import Profiles

In [ ]:
# Create directory structure
profile_dir = Path("../hybrid_renewable_pypsa/data/profiles/grid_profiles")
metadata_path = Path("../hybrid_renewable_pypsa/data/metadata/grid_profiles_metadata.csv")
results_dir = Path("../hybrid_renewable_pypsa/results")
profile_dir.mkdir(parents=True, exist_ok=True)
metadata_path.parent.mkdir(parents=True, exist_ok=True)
results_dir.mkdir(parents=True, exist_ok=True)

def generate_grid_import_profile():
    """Generate grid import profile (p.u.) with demand variation"""
    daily_curve = np.array([
        0.5, 0.4, 0.3, 0.2, 0.2, 0.3, 0.6, 0.8,
        1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.6, 0.7,
        0.8, 0.9, 1.0, 0.9, 0.8, 0.7, 0.6, 0.5
    ])
    base = 1.0  # Normalized to 1.0
    noise_level = 0.05
    profile = np.tile(daily_curve, DAYS)
    noise = np.random.normal(0, noise_level, len(profile))
    return np.clip(base * profile + noise, 0, 1)

# Generate profile
grid_import_profile = generate_grid_import_profile()

# Save grid_import_p_max_pu.csv
grid_import_df = pd.DataFrame({
    "time": TIMES.strftime("%Y-%m-%d %H:%M:%S"),
    "p_max_pu": np.round(grid_import_profile, 4)  # p.u.
})
grid_import_df.to_csv(profile_dir / "grid_import_p_max_pu.csv", index=False)

# Generate metadata file
grid_metadata = pd.DataFrame([
    {"profile_name": "grid_import_p_max_pu.csv", "description": "Grid import profile with daily demand variations.",
     "source": "Synthetic model", "time_resolution": "Hourly", "time_zone": TIME_ZONE}
])
grid_metadata.to_csv(metadata_path, index=False)

print("Successfully generated:")
print("- grid_import_p_max_pu.csv in", profile_dir)
print("- grid_profiles_metadata.csv in", metadata_path)

# Plot grid import profile
plt.figure(figsize=(12, 6))
plt.plot(TIMES, grid_import_profile, label="Grid Import Profile (p.u.)")
plt.title("Grid Import Profile")
plt.xlabel("Time")
plt.ylabel("Power (p.u.)")
plt.legend()
plt.grid(True)

# Save plot
plot_path = results_dir / "grid_import_profile.png"
plt.savefig(plot_path)

plt.show()

print(f"Plot saved to {plot_path}")

## Generator Profiles

In [ ]:
# Create directory structure
profile_dir = Path("../hybrid_renewable_pypsa/data/profiles/generator_profiles")
metadata_path = Path("../hybrid_renewable_pypsa/data/metadata/generator_profiles_metadata.csv")
results_dir = Path("../hybrid_renewable_pypsa/results")
profile_dir.mkdir(parents=True, exist_ok=True)
metadata_path.parent.mkdir(parents=True, exist_ok=True)
results_dir.mkdir(parents=True, exist_ok=True)

def generate_solar_profile():
    """Generate solar generation profile (p.u.) with diurnal pattern"""
    daily_curve = np.array([
        0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.5, 0.7,
        0.9, 1.0, 0.9, 0.7, 0.5, 0.3, 0.1, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
    ])
    base = 1.0  # Max generation is normalized to 1.0
    noise_level = 0.05
    profile = np.tile(daily_curve, DAYS)
    noise = np.random.normal(0, noise_level, len(profile))
    return np.clip(base * profile + noise, 0, 1)

def generate_hydro_profile():
    """Generate hydro generation profile (p.u.) with consistent availability"""
    base = 1.0  # Hydro generation is typically stable
    variation = 0.05  # Minor variations over time
    profile = np.ones(HOURS * DAYS) * base
    noise = np.random.normal(0, variation, len(profile))
    return np.clip(profile + noise, 0, 1)

# Generate profiles
solar_profile = generate_solar_profile()
hydro_profile = generate_hydro_profile()

# Save solar-p_max_pu.csv
solar_df = pd.DataFrame({
    "time": TIMES.strftime("%Y-%m-%d %H:%M:%S"),
    "p_max_pu": np.round(solar_profile, 4)  # p.u.
})
solar_df.to_csv(profile_dir / "solar-p_max_pu.csv", index=False)

# Save hydro-p_max_pu.csv
hydro_df = pd.DataFrame({
    "time": TIMES.strftime("%Y-%m-%d %H:%M:%S"),
    "p_max_pu": np.round(hydro_profile, 4)  # p.u.
})
hydro_df.to_csv(profile_dir / "hydro-p_max_pu.csv", index=False)

# Generate metadata file
generator_metadata = pd.DataFrame([
    {"profile_name": "solar-p_max_pu.csv", "description": "Solar generation profile with diurnal variation.",
     "source": "Synthetic model", "time_resolution": "Hourly", "time_zone": TIME_ZONE},
    {"profile_name": "hydro-p_max_pu.csv", "description": "Hydro generation profile with stable output and minor variations.",
     "source": "Synthetic model", "time_resolution": "Hourly", "time_zone": TIME_ZONE}
])
generator_metadata.to_csv(metadata_path, index=False)

print("Successfully generated:")
print("- solar-p_max_pu.csv in", profile_dir)
print("- hydro-p_max_pu.csv in", profile_dir)
print("- generator_profiles_metadata.csv in", metadata_path)

# Plot generator profiles
plt.figure(figsize=(12, 6))
plt.plot(TIMES, solar_profile, label="Solar Profile (p.u.)")
plt.plot(TIMES, hydro_profile, label="Hydro Profile (p.u.)")
plt.title("Generator Profiles")
plt.xlabel("Time")
plt.ylabel("Power (p.u.)")
plt.legend()
plt.grid(True)

# Save plot
plot_path = results_dir / "generator_profiles.png"
plt.savefig(plot_path)

plt.show()

print(f"Plot saved to {plot_path}")